# Radiator Adaptor for radiator thermostatic valves

Several of the radiators have this type of valve:

https://www.haustechnikdialog.de/Forum/t/161929/Kennt-jemand-diesen-Thermostatkopf-

They use a non-standard M34x1.5 fitting. 


In [1]:
%load_ext autoreload
%autoreload 2
from kadboxes.util import enable_viewer
enable_viewer("Radiator")

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


In [2]:
import cadquery as cq
from cq_warehouse.thread import IsoThread


In [3]:
outer_diameter = 50
inner_diameter = 34

length = 10

thread_depth = 10

f_thread_size = 34.5 # Should be 34 but that is very tight
f_thread_depth = 6

outer_cylinder = cq.Workplane("XY").circle(outer_diameter / 2).extrude(length)
inner_cylinder = cq.Workplane("XY").circle(inner_diameter / 2).extrude(thread_depth)

hollow_cylinder = outer_cylinder.cut(inner_cylinder)

thread = IsoThread(f_thread_size, 1.5, f_thread_depth, external=False, end_finishes=("fade", "fade"))
threaded_cylinder = hollow_cylinder.union(thread)
threaded_cylinder


In [4]:
f_thread_size = 30
f_thread_depth = 6
inner_outer_diameter = 21
inner_diameter = 17

thread = IsoThread(f_thread_size, 1.5, f_thread_depth, external=True, end_finishes=("fade", "fade"))
outer_cylinder = cq.Workplane("XY").circle(thread.min_radius).extrude(f_thread_depth)
screw_cylinder = outer_cylinder.union(thread)

screw_cylinder = screw_cylinder.faces(">Z").circle(inner_outer_diameter / 2).extrude(4)
screw_cylinder = screw_cylinder.faces("<Z").circle(outer_diameter/2).extrude(-2)
inner_cylinder = cq.Workplane("XY").circle(inner_diameter / 2).extrude(30).translate((0,0,-10))
screw_cylinder = screw_cylinder.cut(inner_cylinder)

screw_cylinder = screw_cylinder.translate((0,0,10))

screw_cylinder

In [5]:
inner_fill = cq.Workplane("XY").circle(outer_diameter/2).extrude(5).translate((0,0,5))
inner_fill

In [6]:
# cq.Assembly().add(threaded_cylinder).add(screw_cylinder)
adaptor = threaded_cylinder.union(screw_cylinder).union(inner_fill)

# Cut inner cavity
central_hole = 25
outer_cylinder = cq.Workplane("XY").circle(central_hole/2).extrude(14)

adaptor=adaptor.cut(outer_cylinder)
adaptor

In [7]:
# We try to add some holes to see if that strengthens the construction

hole_count = 18
for rot in range(hole_count):
    angle = rot * 360/hole_count
    wp = cq.Workplane("XY").transformed(rotate=(0, 0, angle)).center(42/2,0)
    wp = wp.circle(1).extrude(100)
    adaptor = adaptor.cut(wp)
adaptor

In [8]:
cq.exporters.export(adaptor, 'radiator-adaptor.stl')
# cq.exporters.export(screw_cylinder, 'screw_cylinder.stl')


# The next part is the cylinder adaptor

In [9]:
central_pin = cq.Workplane("XY").circle((inner_diameter-1)/2).extrude(18)
cq.exporters.export(central_pin, 'radiator-adaptor-central_pin.stl')
central_pin

In [10]:
assembly = cq.Assembly("pin-and-hole") \
  .add(adaptor) \
  .add(central_pin, color=cq.Color("red"), loc=cq.Location(cq.Vector(0, 0, 13))) \

assembly

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.01s
